In [ ]:
import deepxde as dde
import numpy as np


def gen_traindata():
    data = np.load("reaction.npz")
    t, x, ca, cb = data["t"], data["x"], data["Ca"], data["Cb"]
    X, T = np.meshgrid(x, t)
    X = np.reshape(X, (-1, 1))
    T = np.reshape(T, (-1, 1))
    Ca = np.reshape(ca, (-1, 1))
    Cb = np.reshape(cb, (-1, 1))
    return np.hstack((X, T)), Ca, Cb


kf = dde.Variable(0.05)
D = dde.Variable(1.0)


def pde(x, y):
    ca, cb = y[:, 0:1], y[:, 1:2]
    dca_t = dde.grad.jacobian(y, x, i=0, j=1)
    dca_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    dcb_t = dde.grad.jacobian(y, x, i=1, j=1)
    dcb_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
    eq_a = dca_t - 1e-3 * D * dca_xx + kf * ca * cb ** 2
    eq_b = dcb_t - 1e-3 * D * dcb_xx + 2 * kf * ca * cb ** 2
    return [eq_a, eq_b]


def fun_bc(x):
    return 1 - x[:, 0:1]


def fun_init(x):
    return np.exp(-20 * x[:, 0:1])


geom = dde.geometry.Interval(0, 1)
timedomain = dde.geometry.TimeDomain(0, 10)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

bc_a = dde.icbc.DirichletBC(
    geomtime, fun_bc, lambda _, on_boundary: on_boundary, component=0
)
bc_b = dde.icbc.DirichletBC(
    geomtime, fun_bc, lambda _, on_boundary: on_boundary, component=1
)
ic1 = dde.icbc.IC(geomtime, fun_init, lambda _, on_initial: on_initial, component=0)
ic2 = dde.icbc.IC(geomtime, fun_init, lambda _, on_initial: on_initial, component=1)

observe_x, Ca, Cb = gen_traindata()
observe_y1 = dde.icbc.PointSetBC(observe_x, Ca, component=0)
observe_y2 = dde.icbc.PointSetBC(observe_x, Cb, component=1)

data = dde.data.TimePDE(
    geomtime,
    pde,
    [bc_a, bc_b, ic1, ic2, observe_y1, observe_y2],
    num_domain=2000,
    num_boundary=100,
    num_initial=100,
    anchors=observe_x,
    num_test=50000,
)
net = dde.nn.FNN([2] + [20] * 3 + [2], "tanh", "Glorot uniform")

model = dde.Model(data, net)
model.compile("adam", lr=0.001, external_trainable_variables=[kf, D])
variable = dde.callbacks.VariableValue([kf, D], period=1000, filename="variables.dat")
losshistory, train_state = model.train(iterations=80000, callbacks=[variable])
dde.saveplot(losshistory, train_state, issave=True, isplot=True)

C:\Users\kozak\anaconda3\envs\tensorflow-gpu\lib\site-packages\skopt\sampler\sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+2002=2002. 
  warnings.warn("The balance properties of Sobol' points require "
C:\Users\kozak\anaconda3\envs\tensorflow-gpu\lib\site-packages\skopt\sampler\sobol.py:246: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+102=102. 
  warnings.warn("The balance properties of Sobol' points require "
C:\Users\kozak\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\legacy_tf_layers\core.py:236: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
C:\Users\kozak\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\base_layer_v1.py:1676: UserWarning

Compiling model...
Building feed-forward neural network...
'build' took 0.087336 s

'compile' took 2.207980 s

Initializing variables...
Training model...

Step      Train loss                                                                          Test loss                                                                           Test metric
0         [1.54e-02, 9.87e-03, 1.35e+00, 1.28e+00, 1.42e-01, 6.77e-02, 5.13e-01, 4.56e-01]    [1.38e-02, 9.22e-03, 1.35e+00, 1.28e+00, 1.42e-01, 6.77e-02, 5.13e-01, 4.56e-01]    []  
1000      [6.11e-04, 5.38e-04, 1.41e-03, 1.41e-03, 1.83e-03, 2.87e-03, 8.13e-04, 8.93e-04]    [5.23e-04, 4.44e-04, 1.41e-03, 1.41e-03, 1.83e-03, 2.87e-03, 8.13e-04, 8.93e-04]    []  
2000      [1.30e-04, 1.28e-04, 2.15e-04, 2.44e-04, 3.90e-04, 4.18e-04, 1.82e-04, 1.23e-04]    [1.05e-04, 9.20e-05, 2.15e-04, 2.44e-04, 3.90e-04, 4.18e-04, 1.82e-04, 1.23e-04]    []  
3000      [3.98e-05, 4.26e-05, 6.35e-05, 7.27e-05, 1.35e-04, 1.29e-04, 6.61e-05, 1.13e-04]    [3.34e-05, 